# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,May 25 2022 9:25AM,238052,10,0085712193,ISDIN Corporation,Released
1,May 25 2022 9:25AM,238052,10,0085712191,ISDIN Corporation,Released
2,May 25 2022 9:25AM,238052,10,0085712253,ISDIN Corporation,Released
3,May 25 2022 9:25AM,238052,10,0085712295,ISDIN Corporation,Released
4,May 25 2022 9:25AM,238052,10,0085712327,ISDIN Corporation,Released
5,May 25 2022 9:25AM,238052,10,0085712393,ISDIN Corporation,Released
6,May 25 2022 9:25AM,238052,10,0085712394,ISDIN Corporation,Released
7,May 25 2022 9:25AM,238052,10,0085712463,ISDIN Corporation,Released
8,May 25 2022 9:25AM,238052,10,0085712466,ISDIN Corporation,Released
9,May 25 2022 9:25AM,238052,10,0085712497,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
41,238049,Released,54
42,238050,Released,1
43,238051,Released,2
44,238052,Released,42
45,238053,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
238049,NaN,NaN,54.0
238050,NaN,NaN,1.0
238051,NaN,NaN,2.0
238052,NaN,NaN,42.0
238053,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237864,0.0,8.0,3.0
237961,0.0,0.0,1.0
237965,0.0,0.0,1.0
237975,3.0,13.0,0.0
237978,0.0,8.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
237864,0,8,3
237961,0,0,1
237965,0,0,1
237975,3,13,0
237978,0,8,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237864,0,8,3
1,237961,0,0,1
2,237965,0,0,1
3,237975,3,13,0
4,237978,0,8,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,237864,,8,3
1,237961,,,1
2,237965,,,1
3,237975,3,13,
4,237978,,8,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,May 25 2022 9:25AM,238052,10,ISDIN Corporation
42,May 25 2022 9:24AM,238053,10,ISDIN Corporation
43,May 25 2022 9:22AM,238049,10,ISDIN Corporation
97,May 25 2022 9:18AM,238051,10,ISDIN Corporation
99,May 25 2022 9:18AM,238050,22,"NBTY Global, Inc."
100,May 25 2022 9:03AM,238041,15,"Laser Pharmaceuticals, LLC"
118,May 25 2022 9:02AM,238040,15,"Canton Laboratories, LLC"
119,May 25 2022 9:01AM,238039,15,"Carwin Pharmaceutical Associates, LLC"
128,May 25 2022 9:00AM,238038,15,"Chadwick Pharmaceuticals, Inc."
129,May 25 2022 8:59AM,238035,15,"Brookfield Pharmaceuticals, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,May 25 2022 9:25AM,238052,10,ISDIN Corporation,,,42
1,May 25 2022 9:24AM,238053,10,ISDIN Corporation,,,1
2,May 25 2022 9:22AM,238049,10,ISDIN Corporation,,,54
3,May 25 2022 9:18AM,238051,10,ISDIN Corporation,,,2
4,May 25 2022 9:18AM,238050,22,"NBTY Global, Inc.",,,1
5,May 25 2022 9:03AM,238041,15,"Laser Pharmaceuticals, LLC",,,18
6,May 25 2022 9:02AM,238040,15,"Canton Laboratories, LLC",,,1
7,May 25 2022 9:01AM,238039,15,"Carwin Pharmaceutical Associates, LLC",,,9
8,May 25 2022 9:00AM,238038,15,"Chadwick Pharmaceuticals, Inc.",,,1
9,May 25 2022 8:59AM,238035,15,"Brookfield Pharmaceuticals, LLC",,,24


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 25 2022 9:25AM,238052,10,ISDIN Corporation,42,,
1,May 25 2022 9:24AM,238053,10,ISDIN Corporation,1,,
2,May 25 2022 9:22AM,238049,10,ISDIN Corporation,54,,
3,May 25 2022 9:18AM,238051,10,ISDIN Corporation,2,,
4,May 25 2022 9:18AM,238050,22,"NBTY Global, Inc.",1,,
5,May 25 2022 9:03AM,238041,15,"Laser Pharmaceuticals, LLC",18,,
6,May 25 2022 9:02AM,238040,15,"Canton Laboratories, LLC",1,,
7,May 25 2022 9:01AM,238039,15,"Carwin Pharmaceutical Associates, LLC",9,,
8,May 25 2022 9:00AM,238038,15,"Chadwick Pharmaceuticals, Inc.",1,,
9,May 25 2022 8:59AM,238035,15,"Brookfield Pharmaceuticals, LLC",24,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 25 2022 9:25AM,238052,10,ISDIN Corporation,42,,
1,May 25 2022 9:24AM,238053,10,ISDIN Corporation,1,,
2,May 25 2022 9:22AM,238049,10,ISDIN Corporation,54,,
3,May 25 2022 9:18AM,238051,10,ISDIN Corporation,2,,
4,May 25 2022 9:18AM,238050,22,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 25 2022 9:25AM,238052,10,ISDIN Corporation,42.0,NaN,NaN
1,May 25 2022 9:24AM,238053,10,ISDIN Corporation,1.0,NaN,NaN
2,May 25 2022 9:22AM,238049,10,ISDIN Corporation,54.0,NaN,NaN
3,May 25 2022 9:18AM,238051,10,ISDIN Corporation,2.0,NaN,NaN
4,May 25 2022 9:18AM,238050,22,"NBTY Global, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,May 25 2022 9:25AM,238052,10,ISDIN Corporation,42.0,0.0,0.0
1,May 25 2022 9:24AM,238053,10,ISDIN Corporation,1.0,0.0,0.0
2,May 25 2022 9:22AM,238049,10,ISDIN Corporation,54.0,0.0,0.0
3,May 25 2022 9:18AM,238051,10,ISDIN Corporation,2.0,0.0,0.0
4,May 25 2022 9:18AM,238050,22,"NBTY Global, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3094453,208.0,1.0,0.0
15,1428225,67.0,0.0,0.0
16,475986,2.0,0.0,0.0
18,1190034,23.0,1.0,0.0
19,1666096,5.0,22.0,3.0
20,951874,30.0,28.0,10.0
21,237980,1.0,0.0,0.0
22,476080,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3094453,208.0,1.0,0.0
1,15,1428225,67.0,0.0,0.0
2,16,475986,2.0,0.0,0.0
3,18,1190034,23.0,1.0,0.0
4,19,1666096,5.0,22.0,3.0
5,20,951874,30.0,28.0,10.0
6,21,237980,1.0,0.0,0.0
7,22,476080,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,208.0,1.0,0.0
1,15,67.0,0.0,0.0
2,16,2.0,0.0,0.0
3,18,23.0,1.0,0.0
4,19,5.0,22.0,3.0
5,20,30.0,28.0,10.0
6,21,1.0,0.0,0.0
7,22,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,208.0
1,15,Released,67.0
2,16,Released,2.0
3,18,Released,23.0
4,19,Released,5.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20,21,22
Status,,,,,,,,
Completed,0.0,0.0,0.0,0.0,3.0,10.0,0.0,0.0
Executing,1.0,0.0,0.0,1.0,22.0,28.0,0.0,1.0
Released,208.0,67.0,2.0,23.0,5.0,30.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,3.0,10.0,0.0,0.0
1,Executing,1.0,0.0,0.0,1.0,22.0,28.0,0.0,1.0
2,Released,208.0,67.0,2.0,23.0,5.0,30.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20,21,22
0,Completed,0.0,0.0,0.0,0.0,3.0,10.0,0.0,0.0
1,Executing,1.0,0.0,0.0,1.0,22.0,28.0,0.0,1.0
2,Released,208.0,67.0,2.0,23.0,5.0,30.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()